# Self-Attention Mechanism

The self-attention mechanism allows the model to weigh the importance of different words in a sequence. It helps the model focus on relevant words while encoding a particular word. This is a critical part of the Transformer architecture.

In the prior module, We left off with the position encoding embedding. To continue this process, we are pushing the positional embedding into the next compontent of the transformer, the attention mechanism. 

In [1]:
import torch
import torch.nn as nn
import math

# Positional Encoded Embeddings from the previous notebook
pos_encoded_embeddings = torch.tensor([[[ 0.0171,  1.0654,  0.4616,  0.9196,  0.7193, -0.7430,  0.7120,
           0.4198,  2.7427,  0.2844],
         [ 0.9161, -0.4999,  1.8302,  1.2182,  0.0651,  2.0396,  0.3780,
           0.4085,  1.3560, -1.3176],
         [-0.3461,  0.2526,  0.4120,  0.0398,  1.6146,  2.6475, -0.1887,
          -0.6907, -0.4066,  1.2899],
         [-0.8345, -0.4657, -0.5635,  1.1514,  0.1762,  1.8148, -1.4084,
           0.9153, -1.1734,  1.1989],
         [-0.3981, -0.7277, -1.2657,  1.9887, -0.2399,  0.0412,  1.9375,
           0.6083,  0.2095,  1.5739],
         [-1.5240,  1.2359,  0.5596, -0.1529, -0.4064, -0.0906, -1.6746,
           0.2480, -0.2364,  0.8417],
         [ 0.6992, -1.1193,  0.5642,  1.3861, -0.5185,  0.6701, -0.5353,
           1.8013,  0.7900, -0.9430],
         [-0.3550,  2.1032,  1.8690,  0.7174, -1.7692,  0.8200, -0.9620,
           1.8325,  0.3009,  1.0083],
         [ 1.5902, -0.8516,  3.2954, -0.5147,  0.1798,  1.8522, -1.0186,
           0.6484,  0.9932,  0.4030],
         [-0.5990, -0.4916,  0.5419, -0.0293, -0.3465,  1.1548, -1.1130,
          -0.8118,  0.3455,  0.9474]]], dtype=torch.float)

print("Positional Encoded Embeddings Shape:", pos_encoded_embeddings.shape) # torch.Size([1, 10, 10]) because we have 1 batch, 10 tokens, and 10 features
print(pos_encoded_embeddings)

Positional Encoded Embeddings Shape: torch.Size([1, 10, 10])
tensor([[[ 0.0171,  1.0654,  0.4616,  0.9196,  0.7193, -0.7430,  0.7120,
           0.4198,  2.7427,  0.2844],
         [ 0.9161, -0.4999,  1.8302,  1.2182,  0.0651,  2.0396,  0.3780,
           0.4085,  1.3560, -1.3176],
         [-0.3461,  0.2526,  0.4120,  0.0398,  1.6146,  2.6475, -0.1887,
          -0.6907, -0.4066,  1.2899],
         [-0.8345, -0.4657, -0.5635,  1.1514,  0.1762,  1.8148, -1.4084,
           0.9153, -1.1734,  1.1989],
         [-0.3981, -0.7277, -1.2657,  1.9887, -0.2399,  0.0412,  1.9375,
           0.6083,  0.2095,  1.5739],
         [-1.5240,  1.2359,  0.5596, -0.1529, -0.4064, -0.0906, -1.6746,
           0.2480, -0.2364,  0.8417],
         [ 0.6992, -1.1193,  0.5642,  1.3861, -0.5185,  0.6701, -0.5353,
           1.8013,  0.7900, -0.9430],
         [-0.3550,  2.1032,  1.8690,  0.7174, -1.7692,  0.8200, -0.9620,
           1.8325,  0.3009,  1.0083],
         [ 1.5902, -0.8516,  3.2954, -0.5147,  0.17

In [2]:
# Define dimensions for the model
embedding_size = pos_encoded_embeddings.size(2)
# embedding_size is the size of the embeddings vector for each word in the sentence
heads = 2
# heads is the number of attention heads, allowing the model to focus on different parts of the sentence simultaneously
head_dim = embedding_size // heads
# head_dim is the size of each attention head, meaning each head will output a vector of size head_dim ensuring that the embedding size is evenly divisible by the number of heads

print("Embedding Size:", embedding_size)
print("Number of Heads:", heads)
print("Dimension per Head:", head_dim)

Embedding Size: 10
Number of Heads: 2
Dimension per Head: 5


In [8]:
# Define linear transformations for query, key, and value
values_linear = nn.Linear(head_dim, head_dim, bias=False) # Linear layer for values
keys_linear = nn.Linear(head_dim, head_dim, bias=False) # Linear layer for keys
queries_linear = nn.Linear(head_dim, head_dim, bias=False) # Linear layer for queries

# These linear layers project the input embeddings into different vector spaces (query, key, and value)
# This helps in calculating attention scores and obtaining the relevant context for each word

print(values_linear)
print(keys_linear)
print(queries_linear)

Linear(in_features=5, out_features=5, bias=False)
Linear(in_features=5, out_features=5, bias=False)
Linear(in_features=5, out_features=5, bias=False)


In [9]:
# Reshape the input embeddings to split into heads for multi-head attention
N = pos_encoded_embeddings.shape[0] 
print(f"batch size: {N}")
# N is the batch size, which is the number of sentences being processed at once (1 in this case)

value_len, key_len, query_len = pos_encoded_embeddings.shape[1], pos_encoded_embeddings.shape[1], pos_encoded_embeddings.shape[1]
print(f"value len: {value_len}")
print(f"key len: {key_len}")
print(f"query len: {query_len}")
# value_len, key_len, and query_len are the lengths of the input sequences in this case (10 in this case)

values = pos_encoded_embeddings.reshape(N, value_len, heads, head_dim) # Reshape the input embeddings to split into heads for multi-head attention in this case (1, 10, 2, 5).  1 being the batch size, 10 being the number of tokens, 2 being the number of heads, and 5 being the dimension per head
keys = pos_encoded_embeddings.reshape(N, key_len, heads, head_dim) # Reshape the input embeddings to split into heads for multi-head attention in this case (1, 10, 2, 5). 1 being the batch size, 10 being the number of tokens, 2 being the number of heads, and 5 being the dimension per head
queries = pos_encoded_embeddings.reshape(N, query_len, heads, head_dim) # Reshape the input embeddings to split into heads for multi-head attention in this case (1, 10, 2, 5). 1 being the batch size, 10 being the number of tokens, 2 being the number of heads, and 5 being the dimension per head
# The input embeddings are reshaped to separate the heads for multi-head attention

print("Values Shape after Reshape:", values.shape)
print("Keys Shape after Reshape:", keys.shape)
print("Queries Shape after Reshape:", queries.shape)

batch size: 1
value len: 10
key len: 10
query len: 10
Values Shape after Reshape: torch.Size([1, 10, 2, 5])
Keys Shape after Reshape: torch.Size([1, 10, 2, 5])
Queries Shape after Reshape: torch.Size([1, 10, 2, 5])
tensor([[[[ 0.0171,  1.0654,  0.4616,  0.9196,  0.7193],
          [-0.7430,  0.7120,  0.4198,  2.7427,  0.2844]],

         [[ 0.9161, -0.4999,  1.8302,  1.2182,  0.0651],
          [ 2.0396,  0.3780,  0.4085,  1.3560, -1.3176]],

         [[-0.3461,  0.2526,  0.4120,  0.0398,  1.6146],
          [ 2.6475, -0.1887, -0.6907, -0.4066,  1.2899]],

         [[-0.8345, -0.4657, -0.5635,  1.1514,  0.1762],
          [ 1.8148, -1.4084,  0.9153, -1.1734,  1.1989]],

         [[-0.3981, -0.7277, -1.2657,  1.9887, -0.2399],
          [ 0.0412,  1.9375,  0.6083,  0.2095,  1.5739]],

         [[-1.5240,  1.2359,  0.5596, -0.1529, -0.4064],
          [-0.0906, -1.6746,  0.2480, -0.2364,  0.8417]],

         [[ 0.6992, -1.1193,  0.5642,  1.3861, -0.5185],
          [ 0.6701, -0.5353,  1.

In [6]:
# Apply linear transformations to the reshaped embeddings to project them into different vector spaces
values = values_linear(values) # Linear transformation for values
keys = keys_linear(keys) # Linear transformation for keys
queries = queries_linear(queries) # Linear transformation for queries

# The reshaped embeddings are projected into different vector spaces (query, key, and value)
# This helps in calculating attention scores and obtaining the relevant context for each word

print("Values Shape after Linear Transformation:", values.shape)
print("Keys Shape after Linear Transformation:", keys.shape)
print("Queries Shape after Linear Transformation:", queries.shape)

Values Shape after Linear Transformation: torch.Size([1, 10, 2, 5])
Keys Shape after Linear Transformation: torch.Size([1, 10, 2, 5])
Queries Shape after Linear Transformation: torch.Size([1, 10, 2, 5])


In [13]:
# Scaled dot-product attention
d_k = keys.size(-1)
# d_k is the dimension of the keys, used for scaling the dot product to prevent large values

print(f"query matrix: \n {queries} \n")
print(f"key transposed matrix: \n {keys.transpose(-2, 1)} \n") # Transpose the keys matrix to perform matrix multiplication
print(f"square root of d_k: {math.sqrt(d_k)}")


scores = torch.matmul(queries, keys.transpose(-2, -1)) / math.sqrt(d_k)
# scores are calculated by taking the dot product of the queries and keys, and dividing by the square root of teh dimension of the keys

print("Scores Shape:", scores.shape) # In this case, the shape of the scores tensor is (1, 10, 2, 2) 
print("Scores:", scores)

query matrix: 
 tensor([[[[ 0.0171,  1.0654,  0.4616,  0.9196,  0.7193],
          [-0.7430,  0.7120,  0.4198,  2.7427,  0.2844]],

         [[ 0.9161, -0.4999,  1.8302,  1.2182,  0.0651],
          [ 2.0396,  0.3780,  0.4085,  1.3560, -1.3176]],

         [[-0.3461,  0.2526,  0.4120,  0.0398,  1.6146],
          [ 2.6475, -0.1887, -0.6907, -0.4066,  1.2899]],

         [[-0.8345, -0.4657, -0.5635,  1.1514,  0.1762],
          [ 1.8148, -1.4084,  0.9153, -1.1734,  1.1989]],

         [[-0.3981, -0.7277, -1.2657,  1.9887, -0.2399],
          [ 0.0412,  1.9375,  0.6083,  0.2095,  1.5739]],

         [[-1.5240,  1.2359,  0.5596, -0.1529, -0.4064],
          [-0.0906, -1.6746,  0.2480, -0.2364,  0.8417]],

         [[ 0.6992, -1.1193,  0.5642,  1.3861, -0.5185],
          [ 0.6701, -0.5353,  1.8013,  0.7900, -0.9430]],

         [[-0.3550,  2.1032,  1.8690,  0.7174, -1.7692],
          [ 0.8200, -0.9620,  1.8325,  0.3009,  1.0083]],

         [[ 1.5902, -0.8516,  3.2954, -0.5147,  0.1798],

In [11]:
# Apply softmax to get attention weights
attention = torch.nn.functional.softmax(scores, dim=-1)
# The attention weights are calculated using the softmax function to normalize the scores

print("Attention Shape:", attention.shape)
print("Attention Weights:", attention)

Attention Shape: torch.Size([1, 10, 2, 2])
Attention Weights: tensor([[[[0.5016, 0.4984],
          [0.4048, 0.5952]],

         [[0.6133, 0.3867],
          [0.3837, 0.6163]],

         [[0.4321, 0.5679],
          [0.2212, 0.7788]],

         [[0.6546, 0.3454],
          [0.1494, 0.8506]],

         [[0.5803, 0.4197],
          [0.4025, 0.5975]],

         [[0.5431, 0.4569],
          [0.3175, 0.6825]],

         [[0.5607, 0.4393],
          [0.5563, 0.4437]],

         [[0.5641, 0.4359],
          [0.1944, 0.8056]],

         [[0.6179, 0.3821],
          [0.4803, 0.5197]],

         [[0.5039, 0.4961],
          [0.4792, 0.5208]]]], grad_fn=<SoftmaxBackward0>)


In [17]:
# Compute the output as a weighted sum of values
output = torch.matmul(attention, values)
# The output is calculated as the weighted sum of the values, where the weights are the attention weights

print("Output Shape:", output.shape)
print("Output:", output)

Output Shape: torch.Size([1, 10, 2, 5])
Output: tensor([[[[ 5.1275e-01, -2.3062e-02,  7.6786e-01,  3.5396e-01, -4.8948e-01],
          [ 4.7629e-01,  5.5857e-03,  7.1325e-01,  3.1193e-01, -4.6188e-01]],

         [[ 6.8100e-01,  2.7311e-01,  1.4495e-01, -1.7394e-01, -8.7010e-01],
          [ 6.8551e-01,  2.7291e-01,  1.5859e-01, -1.7169e-01, -8.5177e-01]],

         [[-4.1000e-01,  6.4581e-01, -5.4488e-01, -5.8096e-01,  5.3965e-02],
          [-3.2036e-01,  5.2982e-01, -2.9135e-01, -4.3456e-01,  2.2621e-01]],

         [[-2.7894e-02,  2.6303e-01, -7.1920e-02, -2.4139e-01,  2.3254e-01],
          [ 1.0286e-01, -4.0710e-04,  2.4272e-01,  9.5863e-02,  6.4063e-02]],

         [[ 1.7366e-01, -1.0080e-01,  4.3616e-01,  3.5210e-01, -4.7065e-01],
          [ 6.3986e-02,  7.8689e-02,  2.7882e-01,  1.4916e-01, -3.1616e-01]],

         [[ 9.2607e-02, -1.6513e-01,  3.7339e-01,  1.7588e-01,  4.9838e-01],
          [ 9.3334e-02, -1.9488e-01,  3.7261e-01,  1.9396e-01,  4.7630e-01]],

         [[ 8.20

In [18]:
# Combine heads and apply final linear transformation
output = output.transpose(1, 2).contiguous().view(N, -1, heads * head_dim)
# The heads are combined by transposing and reshaping the output to merge the heads

fc_out = nn.Linear(heads * head_dim, embedding_size)
# A final linear layer is applied to project the combined heads back to the original embedding size

output = fc_out(output)

print("Final Output Shape:", output.shape)
print("Final Output:", output)

Final Output Shape: torch.Size([1, 10, 10])
Final Output: tensor([[[ 4.3352e-01,  6.4582e-02, -2.0290e-01, -1.3944e-01, -4.8964e-01,
           6.4467e-03, -2.5833e-01, -4.8392e-02,  1.8122e-01, -4.5369e-01],
         [ 1.9231e-01, -2.8036e-01,  2.4081e-01, -1.3562e-03,  5.1849e-01,
           8.9808e-03,  3.7103e-01, -1.3844e-01, -2.2998e-01, -9.1597e-02],
         [ 2.3325e-01,  5.7716e-02,  3.2724e-01,  3.4075e-02, -1.0252e-01,
          -6.5169e-02,  1.2867e-01, -1.2793e-01, -2.2600e-01,  4.2445e-02],
         [ 5.0482e-01,  1.6524e-02,  5.1617e-02, -7.5409e-02, -3.3103e-01,
          -1.4219e-01, -1.0279e-01, -1.8948e-01,  2.8319e-02, -1.3019e-01],
         [ 4.6103e-01, -1.0390e-01,  3.3408e-01, -2.1866e-01,  7.6646e-02,
          -2.5859e-01,  1.1607e-01, -2.0052e-01, -1.4547e-01,  1.2353e-01],
         [ 4.3236e-01,  5.5618e-02, -1.9086e-01, -1.4108e-01, -4.5655e-01,
           3.0693e-03, -2.3673e-01, -4.7470e-02,  1.7033e-01, -4.4062e-01],
         [ 2.7380e-01, -1.1220e-01, 